<a href="https://colab.research.google.com/github/balasubramaniyan-p/pdsnd_github/blob/master/Index_RSI__R_MACD_Bolinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mftool==2.8

In [2]:
# import module
from mftool import Mftool
obj = Mftool()
scheme_code_dict=obj.get_scheme_codes()

def calculate_rsi(data, window,close="Close",date="Date",format="%d-%m-%Y"):
    # Convert 'Date' column to datetime
    data['Date'] = pd.to_datetime(data[date],format=format)
    data=data.sort_values(by='Date')
    data[close] = data[close].astype(float)
    data = data.reset_index(drop=True)
    delta = data[close].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

  # Find keys by value
list_to_lookup= [
    'quant Flexi Cap Fund - Growth Option-Direct Plan',
    'ICICI Prudential Regular Savings Fund - Direct Plan - Growth',
    'Invesco India PSU Equity Fund - Direct Plan - Growth',
    'ICICI Prudential Income Optimizer Fund (FOF) - Direct Plan - Growth',
    'Axis ELSS Tax Saver Fund - Direct Plan - Growth Option',
    'ICICI Prudential Regular Gold Savings Fund (FOF) - Direct Plan -  Growth',
    'HDFC Equity Savings Fund - Growth Option - Direct Plan',
    'Axis Bluechip Fund - Direct Plan - Growth',
    'ICICI Prudential MNC Fund - Direct Plan - Growth Option',
    'Kotak Equity Savings Fund - Direct - Growth',
    'ICICI Prudential US Bluechip Equity Fund - Direct Plan -  Growth',
    'Axis Gold Fund - Direct Plan - Growth option',
    'Tata Balanced Advantage Fund-Direct Plan-Growth',
    'ICICI Prudential Asset Allocator Fund (FOF) - Direct Plan - Growth',
    'ICICI Prudential Balanced Advantage Fund - Direct Plan -  Growth',
    'ICICI Prudential Nifty Next 50 Index Fund - Direct Plan -  Growth',
    'ICICI Prudential Equity & Debt Fund - Direct Plan - Growth',
    'ICICI Prudential Exports & Services Fund - Direct Plan - Growth',
    'ICICI Prudential Nifty 50 Index Fund - Direct Plan Cumulative Option',
    'ICICI Prudential Focused Equity Fund - Direct Plan - Growth',
    'ICICI Prudential Value Discovery Fund - Direct Plan - Growth',
    'ICICI Prudential Commodities Fund - Direct Plan - Growth Option',
    'quant Multi Asset Fund-GROWTH OPTION-Direct Plan',
    'ICICI Prudential Dividend Yield Equity Fund Direct Plan Growth Option',
    'ICICI Prudential Multi-Asset Fund - Direct Plan - Growth',
    'Nippon India Large Cap Fund - Direct Plan Growth Plan - Growth Option',
    'ICICI Prudential Manufacturing Fund - Direct Plan - Cumulative Option',
    'ICICI Prudential Bluechip Fund - Direct Plan - Growth',
    'ICICI Prudential Smallcap Fund - Direct Plan - Growth',
    'Axis Small Cap Fund - Direct Plan - Growth',
    'ICICI Prudential MidCap Fund - Direct Plan -  Growth',
    'Axis Midcap Fund - Direct Plan - Growth',
    'ICICI PRUDENTIAL HOUSING OPPORTUNITIES FUND - Direct Plan Growth',
    'ICICI Prudential India Opportunities Fund - Direct Plan - Cumulative Option',
    'ICICI Prudential Nifty Alpha Low - Volatility 30 ETF FOF - Direct Plan - Growth',
    'Kotak Bluechip Fund - Growth - Direct',
    'quant Mid Cap Fund - Growth Option - Direct Plan',
    'ICICI Prudential Nifty Alpha Low - Volatility 30 ETF FOF - Direct Plan - IDCW',
    'ICICI Prudential Smallcap Fund - Direct Plan - IDCW',
    'ICICI Prudential Flexicap Fund - Direct Plan - Growth',
    'quant Focused Fund - Growth Option-Direct Plan',
    'ICICI Prudential Multicap Fund - Direct Plan -  Growth',
    'ICICI Prudential MidCap Fund - Direct Plan -  IDCW',
    'ICICI Prudential Infrastructure Fund - Direct Plan -  Growth',
    'quant Small Cap Fund - Growth Option - Direct Plan',
    'ICICI Prudential Nifty Next 50 Index Fund - Direct Plan -  IDCW',
    'Nippon India Small Cap Fund - Direct Plan Growth Plan - Growth Option',
    'Nippon India Growth Fund - Direct Plan Growth Plan - Growth Option',
    'Motilal Oswal Midcap Fund-Direct Plan-Growth Option'
]

tickers = [key for key, value in scheme_code_dict.items() if value in list_to_lookup]
# print(tickers)

# Download historical data for NESI
import pandas as pd
results=[]
for ticker in tickers:
  try:
        data_pre = obj.get_scheme_historical_nav(code=ticker)
        data=pd.DataFrame(data_pre['data'])
        data['RSI'] = calculate_rsi(data, 14,date="date",close="nav",format="%d-%m-%Y")
        # Get the latest RSI value
        latest_rsi = data['RSI'].iloc[-1]
        # print(f"\nThe latest RSI(14) value for {data_pre['scheme_name']} is {latest_rsi}.")
        results.append([data_pre['scheme_name'],latest_rsi])
  except Exception as e:
    print(f"Error occurred while fetching data for {ticker}: {e}")

pd.DataFrame(results,columns=['Scheme','RSI']).sort_values(by='RSI',ascending=True)

,Scheme,RSI
25,Invesco India PSU Equity Fund - Direct Plan - ...,39.402985
28,ICICI Prudential Smallcap Fund - Direct Plan -...,47.817048
27,ICICI Prudential Smallcap Fund - Direct Plan -...,47.867299
30,quant Small Cap Fund - Growth Option - Direct ...,49.008060
24,ICICI Prudential US Bluechip Equity Fund - Dir...,52.680653
29,Nippon India Small Cap Fund - Direct Plan Grow...,55.438363
15,quant Mid Cap Fund - Growth Option - Direct Plan,56.743497
47,ICICI Prudential Nifty Next 50 Index Fund - Di...,57.741502
48,ICICI Prudential Nifty Next 50 Index Fund - Di...,57.741711
5,quant Focused Fund - Growth Option-Direct Plan,58.033073
